# REINFORCE Algorithm

In [1]:
%load_ext autoreload
%autoreload 2

https://medium.com/@thechrisyoon/deriving-policy-gradients-and-implementing-reinforce-f887949bd63

# Instalación de librería gym

https://gym.openai.com/

pip install gym

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gym

![alt_text](reinforcement_diagram.png)

$$\huge max_{\theta} \quad\mathbb{E_{\pi_{\theta}}}\{\sum_{t=0}^{T}R(s_t, a_t)\}$$

Donde R es la recompensa que se recibirá luego de la acción $a_t$ estando en el estado $s_t$

$$\huge R(\tau) = \sum_{t=0}^{T}R(s_t, a_t)\$$

$$\huge J_{\theta} = \sum_{\tau} P(\tau | \theta) R(\tau)$$

$P(\tau, \theta)$ es la probabilidad de una trayectoria dada la policy y contiene el modelo $P(s_i|a_i, s_{i-1})$ y también la policy $\pi_{\theta}(a_i|s_i)$

$$\huge P(\tau| \theta) = P(s_0)\prod_{t=0}^T P(s_{t+1}|a_t, s_{t}) \pi_{\theta}(a_t|s_t)$$

![](Markov_Decision_Process.svg.png)

### Tomando el gradiente

$$\huge  \nabla_{\theta} J_{\theta} =  \sum_{\tau} \nabla_{\theta} P(\tau| \theta) R(\tau)$$

$$\huge \nabla_{\theta} J_{\theta} =  \sum_{\tau} \frac{  P(\tau| \theta)  \nabla_{\theta}  P(\tau| \theta) R(\tau)}{P(\tau, \theta)}$$

$$\huge \nabla_{\theta} J_{\theta} =   \sum_{\tau} P(\tau| \theta)   \nabla_{\theta} log P(\tau| \theta) R(\tau)$$

El gradiente de la policy es una esperanza:



$$\huge \nabla_{\theta} J_{\theta} =   \mathbb{E_{\tau}} \{\nabla_{\theta} log P(\tau| \theta) R(\tau)\}$$


## Montecarlo, unbiased, alta varianza

$$\huge \nabla_{\theta} J_{\theta} = \frac{1}{M} \sum_{i=1}^{M} \nabla_{\theta} log P(\tau_i| \theta) R(\tau_i)$$

$$\huge \nabla_{\theta} J_{\theta} =  \frac{1}{M} \sum_{i=1}^{M} R(\tau_i) \nabla_{\theta}  log [\prod_{t=0}^T P(s_{t+1}^i|a_t^i, s_{t}^i) \pi_{\theta}(a_t^i|s_t^i)]$$

$$\huge  \nabla_{\theta} J_{\theta} =  \frac{1}{M}  \sum_{i=1}^{M} R(\tau_i) \{\sum_{t=0}^T \nabla_{\theta} log P(s_{t+1}^i|a_t^i, s_{t}^i) + \sum_{t=0}^T \nabla_{\theta} log \pi_{\theta}(a_t^i|s_t^i) \}$$

$$\huge \nabla_{\theta} J_{\theta} = \frac{1}{M} \sum_{i=1}^{M}  R(\tau_i)   \sum_{t=0}^T \nabla_{\theta} log \pi_{\theta}(a_t^i|s_t^i)$$

$$ \huge J_{\theta} = \frac{1}{M} \sum_{i=1}^{M}  R(\tau_i) \sum_{t=0}^T  log \pi_{\theta}(a_t^i|s_t^i)$$

# Ejemplo:

Dado un estado $s_t$ entro a la red neuronal y da como resultado de la **softmax**, las probabilidades de las acciones:

[0.1, 0.3, 0.6]

y un reward acumulado de 58

Puedo muestrear o hacer argmax. Supongo lo segundo por lo que puedo plantear que el label correcto (pseudolabel) es [0, 0, 1]

Si aplico **categorical crossentropy**:

J = 0xlog 0.1 + 0xlog 0.3 + 1xlog 0.6

Si re defino el label como R * label: [0, 0, 1]*58

J = 58x0xlog0.1 + 58x0xlog 0.3 + 58x1xlog 0.6

De esta forma podemos resolver el problema con una red neuronal con keras sin demasiado trabajo

# Cart Pole problem

![alt text](cart_pole.gif "Title")

In [3]:
env = gym.make("CartPole-v0")

# REINFORCE. Algoritmo

## 1. Inicializar red neuronal con pesos aleatorios: $\large \pi_{\theta}(a_t|s_t)$
## 2. Correr un episodio y guardar estados ($s_t$), acciones ($a_t$) y rewards ($r_t$)
## 3. Calcular la suma de los discounted rewards:
$$ \huge G_t = \sum_{t'=t+1}^T \gamma^{t'-t-1} r_{t'}$$
## 4. Calcular el gradiente:
$$\huge \nabla_{\theta}J(\theta) = \sum_{t=0}^{T-1}{\nabla_{\theta} log \pi_{\theta}}(a_t|s_t)G_t$$
## 5. Tomar un paso en dirección del gradiente (Queremos maximizar)
$$ \huge \theta = \theta + \alpha \nabla_{\theta}J(\theta) $$
## 6. Repetir desde el paso 2 hasta convergencia

# Toy example

In [4]:
actions = [[1., 0.], [0., 1.], [0, 1], [1, 0]]
pi_outs = [[.9, .1], [.4, .6], [0.2, 0.8], [0.3, 0.7]]
discounted_rewards = [12.24, 11.36, 10.466, 9.56]

In [5]:
import tensorflow as tf
from tensorflow.keras import backend as K
actions = tf.constant(actions, shape=[4,2])
pi = tf.constant(pi_outs, shape=[4,2])
discounted_rewards = tf.constant(discounted_rewards, shape=[4,1])
product = discounted_rewards*actions
loss = K.categorical_crossentropy(product, pi)

In [6]:
actions_np = K.eval(actions)
pi_np = K.eval(pi)
discounted_rewards_np = K.eval(discounted_rewards)
product_np = K.eval(product)
loss_np = K.eval(loss)
print(actions_np)
print(pi_np)
print(discounted_rewards_np)
print(product_np)
print(loss_np)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[[0.9 0.1]
 [0.4 0.6]
 [0.2 0.8]
 [0.3 0.7]]
[[12.24 ]
 [11.36 ]
 [10.466]
 [ 9.56 ]]
[[12.24   0.   ]
 [ 0.    11.36 ]
 [ 0.    10.466]
 [ 9.56   0.   ]]
[ 1.289613   5.8029785  2.3354201 11.50998  ]


In [7]:
np.log(0.9)*12.24, np.log(0.6)*11.36, np.log(0.8)*10.466, np.log(0.3)*9.56

(-1.2896127116517937,
 -5.802979085981654,
 -2.3354204080545187,
 -11.50998000935595)

# Train full model

In [8]:
from reinforce_alg_helper import plot_episode, actions_to_one_hot, get_policy_model, apply_baselines, discount_rewards, score_model, run_episode, get_observations_stats, get_random_episode, get_batch_data

Using TensorFlow backend.


In [9]:
from keras.layers import Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
import keras.backend as K
from keras.initializers import glorot_uniform
from keras.losses import categorical_crossentropy, binary_crossentropy

def get_policy_model_softmax(lr=0.1, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
    model = Sequential()
    model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(Adam(lr), loss=['categorical_crossentropy'])
    return model

In [10]:
model = get_policy_model_softmax()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 898
Trainable params: 898
Non-trainable params: 0
_________________________________________________________________


In [100]:
def run_episode_with_policy(env, model, exploit = False):
    states = []
    actions = []
    rewards = []
    predictions = []
    observation = env.reset()
    done = False
    episode_length = 0
    while not done:
        (position_of_cart, velocity_of_cart, angle_of_pole, rotation_rate_of_pole) = observation
        state = observation.reshape(1,-1)
        states.append(observation)
        predict = model.predict([state])[0]
        predictions.append(predict)
        if exploit:
            #exploit
            action = np.argmax(predict)
        else:
            #Explore
            action = np.random.choice(range(len(predict)),p=predict)
        actions.append(action)
        observation, reward, done, _ = env.step(action)
        reward = (env.observation_space.high[2]/4 - np.abs(angle_of_pole))*180/np.pi
        if done: reward-=12
        rewards.append(reward)
        episode_length += 1
    return np.array(states), np.array(actions), np.array(rewards), episode_length, np.array(predictions)

In [101]:
states, actions, rewards, episode_length, predictions = run_episode_with_policy(env, model)

In [102]:
def get_discounted_rewards(r, gamma = 0.999):
    # Por si es una lista
    r = np.array(r, dtype=float)
    """Take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [103]:
#states

In [104]:
actions

array([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0])

In [105]:
rewards

array([ 4.42213879e+00,  4.37920120e+00,  4.66155295e+00,  4.59844579e+00,
        4.86217896e+00,  5.45234633e+00,  5.70002917e+00,  5.60888463e+00,
        5.85123192e+00,  5.75573076e+00,  5.32390222e+00,  5.22591975e+00,
        4.78830805e+00,  4.68117008e+00,  4.23108321e+00,  3.43739312e+00,
        2.96777483e+00,  2.14700067e+00,  1.64200252e+00,  7.78217844e-01,
       -4.47235577e-01, -1.37160477e+00, -2.00345109e+00, -2.34934748e+00,
       -3.07714387e+00, -3.52593852e+00, -3.70104144e+00, -3.60574914e+00,
       -3.24134391e+00, -2.60710410e+00, -2.36114953e+00, -1.83813375e+00,
       -1.03624789e+00, -6.15295582e-01,  9.39428425e-02,  4.28809964e-01,
        3.93166430e-01, -1.11285708e-02, -1.17150313e-01,  7.22760324e-02,
        5.56415190e-01,  6.69939046e-01,  4.15530695e-01, -2.06163337e-01,
       -1.19653580e+00, -2.55896568e+00, -4.29870807e+00, -1.77604859e+01])

In [106]:
predictions

array([[0.6014472 , 0.3985528 ],
       [0.40594658, 0.5940534 ],
       [0.6037375 , 0.39626253],
       [0.4094982 , 0.59050184],
       [0.22768736, 0.77231264],
       [0.38935626, 0.61064374],
       [0.5860581 , 0.41394192],
       [0.38615194, 0.61384803],
       [0.5826964 , 0.41730356],
       [0.66544366, 0.3345563 ],
       [0.5974602 , 0.40253976],
       [0.67161596, 0.32838404],
       [0.6146416 , 0.38535833],
       [0.6771497 , 0.32285026],
       [0.65635496, 0.34364498],
       [0.6859016 , 0.31409845],
       [0.6593123 , 0.34068766],
       [0.693511  , 0.306489  ],
       [0.6589541 , 0.34104592],
       [0.5904506 , 0.4095494 ],
       [0.6568329 , 0.34316716],
       [0.70321685, 0.29678312],
       [0.71257186, 0.28742817],
       [0.6958822 , 0.30411783],
       [0.71671414, 0.2832859 ],
       [0.7133471 , 0.28665292],
       [0.66721004, 0.33278987],
       [0.5540392 , 0.44596082],
       [0.40243676, 0.5975632 ],
       [0.55865955, 0.4413405 ],
       [0.

In [107]:
get_discounted_rewards(rewards)

array([ 32.41864489,  28.02453063,  23.66899843,  19.02647195,
        14.44246863,   9.58987955,   4.14167489,  -1.55991419,
        -7.17597479, -13.04024696, -18.81479251, -24.16285759,
       -29.41819554, -34.24074434, -38.96087529, -43.2351937 ,
       -46.71930612, -49.73681777, -51.93575419, -53.63138811,
       -54.46407002, -54.07090535, -52.75205264, -50.79940095,
       -48.49855202, -45.46687502, -41.98291942, -38.32019818,
       -34.74919824, -31.53939372, -28.96125087, -26.62672807,
       -24.81340773, -23.8009608 , -23.20887409, -23.32614308,
       -23.77873178, -24.1960943 , -24.2091749 , -24.11614073,
       -24.21262939, -24.79383842, -25.48926674, -25.93072816,
       -25.75031514, -24.57835769, -22.04143344, -17.76048586])

In [108]:
actions_to_one_hot(actions)

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [109]:
class RunningVariance:
    # Keeps a running estimate of variance

    def __init__(self):
        self.m_k = None
        self.s_k = None
        self.k = None

    def add(self, x):
        if not self.m_k:
            self.m_k = x
            self.s_k = 0
            self.k = 0
        else:
            old_mk = self.m_k
            self.k += 1
            self.m_k += (x - self.m_k) / self.k
            self.s_k += (x - old_mk) * (x - self.m_k)

    def get_variance(self):
        return self.s_k / (self.k - 1)

In [115]:
running_variance = RunningVariance()
goal = 500
epsilon=1e-12
model = get_policy_model_softmax(lr=0.01) #0.001
print_every = 20
loss_array = []
discounted_rewards_sum = []
steps = []
scores = []
rew_sum = 0
gamma=0.999
for num_episode in range(2000):
    states, actions, rewards, episode_length, probs = run_episode_with_policy(env, model)
    discounted_rewards = get_discounted_rewards(rewards, gamma=gamma)
    rew_sum = rew_sum + rewards.sum()
    probs = np.array(probs)
    # discounted_rewards_normalized = (discounted_rewards.reshape(-1, 1) - discounted_rewards.mean())/discounted_rewards.std()
    discounted_rewards_normalized = discounted_rewards.reshape(-1, 1)
    for dr in discounted_rewards_normalized:
        running_variance.add(dr[0])
    pseudolabels = actions_to_one_hot(actions)*discounted_rewards_normalized
    history = model.fit(states, pseudolabels, verbose=0) #batch_size=len(states)
    loss = history.history['loss'][0]
    loss_array.append(loss)
    discounted_rewards_sum.append(discounted_rewards[0])
    steps.append(len(states))
    # score = score_model(model, env, 10)
    entropy = np.mean(-np.sum(np.log(probs+epsilon)*probs, axis=1)/np.log(2))
    rv = running_variance.get_variance()
    if (num_episode%print_every) == 0:
        score = rew_sum/print_every
        scores.append(score)
        print(f'{num_episode}) episode length: {len(states)} loss: {loss:.3f}, score: {score}, entropy: {entropy:.3f}, running_var: {rv}')
        rew_sum = 0
    if score>=goal:
        print(f'Goal Reached in {num_episode} episodes! Final score:', score)
        break

0) episode length: 20 loss: -16.055, score: 0.45138707895045604, entropy: 1.000, running_var: 167.8182246374874
20) episode length: 18 loss: -6.097, score: 32.569279816274815, entropy: 0.898, running_var: 925.7793653950125
40) episode length: 46 loss: -19.972, score: 52.325048808348285, entropy: 0.747, running_var: 1709.1453919575197
60) episode length: 90 loss: 31.949, score: 81.60682658729482, entropy: 0.792, running_var: 3139.137072113438
80) episode length: 26 loss: -23.338, score: 155.5672765971031, entropy: 0.806, running_var: 12139.152541115673
100) episode length: 55 loss: -30.661, score: 110.15726707678486, entropy: 0.591, running_var: 11143.463846172845
120) episode length: 77 loss: -4.209, score: 114.5970697956941, entropy: 0.444, running_var: 10621.227081269271
140) episode length: 53 loss: 1.901, score: 192.98286181721363, entropy: 0.348, running_var: 17404.399976312547
160) episode length: 36 loss: -4.983, score: 38.376394602532514, entropy: 0.279, running_var: 16291.8234

KeyboardInterrupt: 

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(20,8))
ax = ax.flatten()
ax[0].set_title('loss normalizada')
ax[0].plot(np.array(loss_array)/np.array(steps).reshape(-1))
ax[1].set_title('discount_rewards_sum')
ax[1].plot(discounted_rewards_sum)
ax[2].set_title('steps')
ax[2].plot(steps)
ax[3].set_title('Score')
ax[3].plot(scores)
plt.show()

In [ ]:
states, actions, rewards, reward_sum, discounted_rewards = run_episode(env, model, greedy=True)
plot_episode(*states.T, actions, show_pos_thres=True)

In [ ]:
%time states_means, states_stds = get_observations_stats(env, lambda env: run_episode(env, model, greedy=True), N=1000)

In [ ]:
%time states_means, states_stds = get_observations_stats(env, lambda env: run_episode(env, model, greedy=False), N=1000)

# Con batches

In [ ]:
def train_full_model(lr=0.1, max_num_episodes = 10000, episodes_batch_size = 50, training_epochs = 1, goal = 200, 
                     reset_model=True, hidden_layer_neurons = 128, verbose_period = 2, 
                     score_thres=10, states_means=None, states_stds=None,
                     model_train=None, model_predict=None, epsilon=1e-12):
    losses=[]
    if model_train is None:
        # Get model
        model_train, model_predict = get_policy_model(env, hidden_layer_neurons, lr)
    num_episode = 0
    i = 0
    
    while num_episode < max_num_episodes:
        # Get batch_size episodes for training
        batch_states, batch_actions, discounted_rewards, batch_probs = get_batch_data(env, model_predict, episodes_batch_size)
        if states_means is not None:
            batch_states = (batch_states - states_means)/states_stds
        # format data for NN
        discounted_rewards = apply_baselines(discounted_rewards)
        # discounted_rewards = np.ones((len(batch_actions), 1))*len(batch_states)/episodes_batch_size
        actions_train = actions_to_one_hot(batch_actions)
        hist = model_train.fit([batch_states, discounted_rewards], actions_train, 
                               batch_size=len(batch_states),
                               epochs=training_epochs, verbose=0)
        loss = hist.history['loss'][0]
        losses.append(loss)
        score = score_model(model_predict, env, score_thres)
        if (i%verbose_period) == 0:
            entropy = np.mean(-np.sum(np.log(batch_probs+epsilon)*batch_probs, axis=1)/np.log(2))
            print(f'{num_episode}) episode avg_len: {len(batch_states)/episodes_batch_size} loss: {loss:.3f}, score: {score}, entropy: {entropy:.3f}')
            # print(num_episode, np.mean(losses), score, entropy)
        if score >= goal:
            print("Solved in {} episodes!".format(num_episode))
            break
        num_episode+=episodes_batch_size
        i+=1
    return losses, model_train, model_predict

In [ ]:
losses, model_train, model_predict = train_full_model(lr=0.001, training_epochs = 1, hidden_layer_neurons = 128, 
                                                      episodes_batch_size=1, max_num_episodes=10000, 
                                                      score_thres = 10, 
                                                      verbose_period = 10,
                                                      #states_means=states_means, states_stds=states_stds,
                                                      model_train=None, model_predict=None)
#                                                       model_train=model_train, model_predict=model_predict)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(losses)

In [ ]:
states, actions, rewards, reward_sum, discounted_rewards = run_episode(env, model_predict, greedy=True)
plot_episode(*states.T, actions, show_pos_thres=True)

In [ ]:
%time states_means, states_stds = get_observations_stats(env, lambda env: run_episode(env, model_predict, greedy=True), N=1000)

In [ ]:
%time states_means, states_stds = get_observations_stats(env, lambda env: run_episode(env, model_predict, greedy=False), N=1000)

In [ ]:
# model_predict.save('model_predict-lr_0.005-training_epochs_1-hidden_layer_neurons_128-batch_size_50.hdf5')
# model_train.save('model_train-lr_0.005-training_epochs_1-hidden_layer_neurons_128-batch_size_50.hdf5')

In [ ]:
print_every

In [ ]:
training_episodes = 1
reward_sum = 0
model_train, model_predict = get_policy_model(env, hidden_layer_neurons, 0.01)
num_actions = env.action_space.n

# Placeholders for our observations, outputs and rewards
states = np.empty(0).reshape(0,dimen)
actions = np.empty(0).reshape(0,1)
rewards = np.empty(0).reshape(0,1)
discounted_rewards = np.empty(0).reshape(0,1)

# Setting up our environment
observation = env.reset()

num_episode = 0

losses = []

while num_episode < num_episodes:
    # Append the observations to our batch
    state = np.reshape(observation, [1, dimen])
    
    predict = model_predict.predict([state])[0]
    action = np.random.choice(range(num_actions),p=predict)
    
    # Append the observations and outputs for learning
    states = np.vstack([states, state])
    actions = np.vstack([actions, action])
    
    # Determine the oucome of our action
    observation, reward, done, _ = env.step(action)
    reward_sum += reward
    rewards = np.vstack([rewards, reward])
    
    if done:
        # Determine standardized rewards
        discounted_rewards_episode = discount_rewards(rewards, gamma)       
        discounted_rewards = np.vstack([discounted_rewards, discounted_rewards_episode])
        
        rewards = np.empty(0).reshape(0,1)

        if (num_episode + 1) % batch_size == 0:
            discounted_rewards -= discounted_rewards.mean()
            discounted_rewards /= discounted_rewards.std()
            discounted_rewards = discounted_rewards.squeeze()
            actions = actions.squeeze().astype(int)
           
            actions_train = np.zeros([len(actions), num_actions])
            actions_train[np.arange(len(actions)), actions] = 1
            for i in range(training_episodes):
                loss = model_train.train_on_batch([states, discounted_rewards], actions_train)
            losses.append(loss)

            # Clear out game variables
            states = np.empty(0).reshape(0,dimen)
            actions = np.empty(0).reshape(0,1)
            discounted_rewards = np.empty(0).reshape(0,1)


        # Print periodically
        if (num_episode + 1) % print_every == 0:
            # Print status
            score = score_model(model_predict,10)
            print("Average reward for training episode {}: {:0.2f} Test Score: {:0.2f} Loss: {:0.6f}".format(
                (num_episode + 1), reward_sum/print_every, 
                score,
                np.mean(losses[-print_every:],)
            ))
            
            if score >= goal:
                print("Solved in {} episodes!".format(num_episode))
                break
            reward_sum = 0
                
        num_episode += 1
        observation = env.reset()
        

In [ ]:
plt.plot(losses)

In [ ]:
env.close()